In [1]:
import cv2
import numpy as np

In [2]:
##stack images 
## TO STACK ALL THE IMAGES IN ONE WINDOW



def stackImages(imgArray,scale,lables=[]):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
            hor_con[x] = np.concatenate(imgArray[x])
        ver = np.vstack(hor)
        ver_con = np.concatenate(hor)
    else:
        for x in range(0, rows):
            imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        hor_con= np.concatenate(imgArray)
        ver = hor
    if len(lables) != 0:
        eachImgWidth= int(ver.shape[1] / cols)
        eachImgHeight = int(ver.shape[0] / rows)
        #print(eachImgHeight)
        for d in range(0, rows):
            for c in range (0,cols):
                cv2.rectangle(ver,(c*eachImgWidth,eachImgHeight*d),(c*eachImgWidth+len(lables[d][c])*13+27,30+eachImgHeight*d),(255,255,255),cv2.FILLED)
                cv2.putText(ver,lables[d][c],(eachImgWidth*c+10,eachImgHeight*d+20),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,255),2)
    return ver



#to find the contour of the rectangle by finding the four contours of the image and then finding the biggest and the small and so on
def rectContour(contours):

    rectCon = []
    max_area = 0 #neglecting smaller contours to find out the biggest mcq box and grade card
    for i in contours:
        area = cv2.contourArea(i)
        #print("Area",area)
        if area > 50:
            peri = cv2.arcLength(i, True) #total lenght of the contour , here true means that we want a closed contour 
            approx = cv2.approxPolyDP(i, 0.02 * peri, True) #what type of polygon it is ie. how many corner point it has
            #print("corner points", len(approx))
            if len(approx) == 4:
                rectCon.append(i) #list contour points of the rectangle
    rectCon = sorted(rectCon, key=cv2.contourArea,reverse=True) #sorting it out on area #DESCENDING ORDER IE  BIGGER FIRST HENCE REVERRSE IS TRUE
    #print(len(rectCon))
    return rectCon


#THIS IS TO GET THE CORNER POINTS
def getCornerPoints(cont):
    peri = cv2.arcLength(cont, True) # LENGTH OF CONTOUR
    approx = cv2.approxPolyDP(cont, 0.02 * peri, True) # APPROXIMATE THE POLY TO GET CORNER POINTS
    return approx




def reorder(myPoints):

    myPoints = myPoints.reshape((4, 2)) # REMOVE EXTRA BRACKET
    print(myPoints)
    myPointsNew = np.zeros((4, 1, 2), np.int32) # NEW MATRIX WITH ARRANGED POINTS
    add = myPoints.sum(1)
    print(add)
    print(np.argmax(add))
    myPointsNew[0] = myPoints[np.argmin(add)]  #[0,0]
    myPointsNew[3] =myPoints[np.argmax(add)]   #[w,h]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]  #[w,0]
    myPointsNew[2] = myPoints[np.argmax(diff)] #[h,0]

    return myPointsNew

def splitBoxes(img):
    rows = np.vsplit(img,5) #TO GET THE ROWS AND THE NUMBER OF ROWS
    boxes=[]
    for r in rows:
        cols= np.hsplit(r,5)
        for box in cols:
            boxes.append(box)
            #cv2.imshow("split",box)
    return boxes

#MARKING RED OR GREEN 
def showAnswers(img,myIndex,grading,ans,questions=5,choices=5):
    secW = int(img.shape[1]/questions)
    secH = int(img.shape[0]/choices)
    # A 100X100 BOX IS CREATED
   #FINDING THE CENTRE 
    for x in range(0,questions):
        myAns = myIndex[x]
        cX = (myAns * secW) + secW // 2 #to get to the center horizontally
        cY = (x * secH) + secH // 2  #to get to the center vertically
        
    #creating the circle and marking red or green
        if grading[x]==1:
            myColor = (0,255,0)
            #cv2.rectangle(img,(myAns*secW,x*secH),((myAns*secW)+secW,(x*secH)+secH),myColor,cv2.FILLED)
            cv2.circle(img,(cX,cY),50,myColor,cv2.FILLED)
        else:
            myColor = (0,0,255)
            #cv2.rectangle(img, (myAns * secW, x * secH), ((myAns * secW) + secW, (x * secH) + secH), myColor, cv2.FILLED)
            cv2.circle(img, (cX, cY), 50, myColor, cv2.FILLED)

            # CORRECT ANSWER
            myColor = (0, 255, 0)
            correctAns = ans[x]
            cv2.circle(img,((correctAns * secW)+secW//2, (x * secH)+secH//2),
            20,myColor,cv2.FILLED)

    return img



In [3]:
path = "1.jpg"
widthIMG = 700
heightIMG = 700
questions = 5
choices = 5
ans = [1,2,0,1,4]
webcamFeed = True
cameraNo = 0 # means one default camera and  1 means that multiple cameras

In [ ]:
#WEB CAM ACCESS

cap = cv2.VideoCapture(cameraNo)
cap.set(10,150)

while True:
    if webcamFeed:
        success, img = cap.read()
    else:
        img = cv2.imread(path)
            
            
            
            
            
            
            
    #Image Preprocessing
    img = cv2.resize(img,(widthIMG,heightIMG)) #resize
    imgContours = img.copy()
    imgBiggestContours = img.copy()
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #add gausian blur
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
    #edge detection
    imgCanny = cv2.Canny(imgBlur,10,50)

    try:
        #FINDING ALL CONTOURS
        contours, hierarchy = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # FIND ALL CONTOURS
        cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 10) # DRAW ALL DETECTED CONTOURS #green countors 
        #now we have both the green countor which detects the grade box as well as detects the mcq box


        #FIND JUST THE RECTANGLES 
        rectCon = rectContour(contours)
        #now to find biggest countor of the image
        biggestContour = getCornerPoints(rectCon[0])
        #print(biggestContour.shape)
        #found thhe biggest rectangle above hence rectCOn 0
        gradePoints = getCornerPoints(rectCon[1])
        #print(len(biggestContour))

        if biggestContour.size != 0 and gradePoints.size != 0:
            cv2.drawContours(imgBiggestContours,biggestContour,-1,(0, 255, 0), 20)
            cv2.drawContours(imgBiggestContours,gradePoints,-1,(255, 0, 0), 20)
            biggestContour = reorder(biggestContour) 
            gradePoints = reorder(gradePoints)
            #birds eye view OF THE BIGGEST RECTANGLE 
            pt1 = np.float32(biggestContour)
            pt2 = np.float32([[0,0],[widthIMG,0],[0,heightIMG],[widthIMG,heightIMG]]) #prepare points for warp
            matrix = cv2.getPerspectiveTransform(pt1,pt2)
            imgWarpColored = cv2.warpPerspective(img, matrix, (widthIMG, heightIMG)) # APPLY WARP PERSPECTIVE
            #this will the trasnformation matriX


            ptG1 = np.float32(gradePoints)
            ptG2 = np.float32([[0,0],[325,0],[0,150],[325,150]]) #prepare points for warp
            matrixG = cv2.getPerspectiveTransform(ptG1,ptG2)
            imgGradeDisplay = cv2.warpPerspective(img, matrixG, (325, 150)) # APPLY WARP PERSP
            #cv2.imshow("Grade",imgGradeDisplay)

        #NOW WE WILLL BE ABLE TO DETECT THE BIGGEST CONTOURS

        #using the logic of pixels i.e. the number of non zero pixels are less in the non marked circle
        #and the number of non zero pixels is more in the marked circle

        #HENCE APPLYING THRESHOLD
            imgWarpGray = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY) # CONVERT TO GRAYSCALE
            imgThresh = cv2.threshold(imgWarpGray, 170, 255,cv2.THRESH_BINARY_INV )[1] # APPLY THRESHOLD AND INVERSE
            #THIS CONVERTS THE IMAGE FROM GRAY SCALE TO A BINARY SCALE 
            boxes = splitBoxes(imgThresh)
           # cv2.imshow("Test",boxes[2])
            #the boxes ie the circles are numbered as [[0,1,2,3,4],
            #                                          [5,6,7,8,9]
             #                                         [10,11,12,13,14]          
               #                                       [15,16,17,18,19]
            #                                          [20,21,22,23,24]]

            #

            #TO FIND IF MARKED OR NOT WE ARE GONNA FIND THE NON ZERO IMAGES 
            #print(cv2.countNonZero(boxes[1]),cv2.countNonZero(boxes[2]))
            #10425-marked 10k non zero     2818-nonmarked 2k non zero


         #GETTING NON ZERO PIXEL VALUES OF EACH BOCX
            myPixelVal = np.zeros((questions,choices))
            countC = 0
            countR =0
            for image in boxes:
                totalPixels = cv2.countNonZero(image)
                myPixelVal[countR][countC] = totalPixels
                countC +=1 
                if (countC== choices):countR +=1 ; countC=0
            print(myPixelVal)



            #FINDING INDEX VALUES OF THE MARKINGS
            myIndex = []
            for x in range (0,questions):
                arr = myPixelVal[x]
                print("arr",arr)
                myIndexVal = np.where(arr==np.amax(arr))
                #print(myIndexVal[0])
                myIndex.append(myIndexVal[0][0])
            #print(myIndex)


            #GRADING
            grading = []
            for x in range(0,questions):
                if ans[x] == myIndex[x]:
                    grading.append(1)
                else:
                    grading.append(0)
            #print(grading)
            score = (sum(grading)/questions) *100 #final grade
            print(score)



            #DISPLAYING ANSWERS
            imgResult = imgWarpColored.copy()
            imgResult =showAnswers(imgResult,myIndex,grading,ans,questions,choices)
            #cv2.imshow("answer",imgResult)
            imgRawDrawing=  np.zeros_like(imgWarpColored)
            imgRawDrawing =showAnswers(imgRawDrawing,myIndex,grading,ans,questions,choices)
            invMatrix = cv2.getPerspectiveTransform(pt2,pt1)
            imgInvWarp = cv2.warpPerspective(imgRawDrawing,invMatrix, (widthIMG, heightIMG))

            #DISPLAY GRADE
            imgRawGrade = np.zeros_like(imgGradeDisplay) #NEW BLANK IMAGE WITH GRADE AREA SIZE
            cv2.putText(imgRawGrade,str(int(score))+"%",(70,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,255),3)
            # ADDED THE GRADE TO NEW IMAGE
            invMatrixG = cv2.getPerspectiveTransform(ptG2,ptG1)
            imgInvGradeDisplay = cv2.warpPerspective(imgRawGrade,invMatrixG, (widthIMG, heightIMG))
            #cv2.imshow("Grade",imgRawGrade)


            #SHOW ANSWERS AND GRADE ON THE FINAL IMAGE
            imgFinal = img.copy()
            imgFinal = cv2.addWeighted(imgFinal,1,imgInvWarp,1,0)
            imgFinal = cv2.addWeighted(imgFinal,1, imgInvGradeDisplay,1,0)



        imgBlank = np.zeros_like(img) #creating a blank image 
        imageArray = ([img,imgGray,imgBlur,imgCanny],
                     [imgContours,imgBiggestContours,imgWarpColored,imgThresh],
                     [imgResult,imgRawDrawing,imgInvWarp,imgFinal])
    except:
        imageArray = ([img,imgGray,imgCanny,imgContours],
                      [imgBlank, imgBlank, imgBlank, imgBlank])
    #stack images function used
    lables  = [["Orignal","Gray","Blur","Canny"],
               ["Contours","BiggestContour","Warp","Threshold"],
              ["Result","RawDrawing","InvWarp","Final Image"]]
    imgStacked = stackImages(imageArray,0.3,lables)
    cv2.imshow("final Result",imgFinal)


    cv2.imshow("Stacked Images",imgStacked)
    if cv2.waitKey(1) & 0xFF == ord('s'):
        cv2.imwrite("Scanned/myImage"+str(count)+".jpg",imgFinal)
        cv2.rectangle(stackedImage, ((int(stackedImage.shape[1] / 2) - 230), int(stackedImage.shape[0] / 2) + 50),
                      (1100, 350), (0, 255, 0), cv2.FILLED)
        cv2.putText(stackedImage, "Scan Saved", (int(stackedImage.shape[1] / 2) - 200, int(stackedImage.shape[0] / 2)),
                    cv2.FONT_HERSHEY_DUPLEX, 3, (0, 0, 255), 5, cv2.LINE_AA)
        cv2.imshow('Result', stackedImage)
        cv2.waitKey(300)
        count += 1
    

[[209 425]
 [245 455]
 [254 450]
 [218 418]]
[634 700 704 636]
2
[[  1 185]
 [ 53 234]
 [ 71 261]
 [ 53 233]]
[186 287 332 286]
2
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[ 32 313]
 [208 439]
 [202 569]
 [208 438]]
[345 647 771 646]
2
[[188 695]
 [186 626]
 [  0 635]
 [185 626]]
[883 812 635 811]
0
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[486 293]
 [476 311]
 [477 406]
 [486 392]]
[779 787 883 878]
2
[[231 456]
 [231 461]
 [259 480]
 [262 479]]
[687 692 739 741]
3
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[667 5

[[ 84 209]
 [ 50 354]
 [ 76 272]
 [ 80 354]]
[293 404 348 434]
3
[[ 52 519]
 [ 27 518]
 [  3 699]
 [ 20 538]]
[571 545 702 558]
2
[[19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
20.0
[[39 44]
 [52 74]
 [88 78]
 [77 47]]
[ 83 126 166 124]
2
[[ 69 314]
 [ 47 339]
 [ 81 350]
 [ 65 508]]
[383 386 431 573]
3
[[19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 

[[412 279]
 [507 549]
 [412 286]
 [588 349]]
[ 691 1056  698  937]
1
[[562 485]
 [601 653]
 [530 543]
 [603 665]]
[1047 1254 1073 1268]
3
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[349 269]
 [346 539]
 [259 541]
 [346 540]]
[618 885 800 886]
3
[[417 302]
 [449 475]
 [452 314]
 [447 426]]
[719 924 766 873]
1
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[166 378]
 [ 64 666]
 [156 408]
 [167 488]]
[544 730 564 655]
1
[[563 488]
 [601 654]
 [544 583]
 [603 666]]
[1051 1255 1127 1269]
3
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]


[[442 392]
 [437 509]
 [448 412]
 [462 521]]
[834 946 860 983]
3
[[  0 210]
 [163 299]
 [139 500]
 [164 299]]
[210 462 639 463]
2
[[19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
20.0
[[630 234]
 [621 307]
 [629 282]
 [629 379]]
[ 864  928  911 1008]
3
[[  0 211]
 [163 301]
 [139 500]
 [163 300]]
[211 464 639 463]
2
[[19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19

[[249  84]
 [417 199]
 [248  84]
 [248 364]]
[333 616 332 612]
1
[[311 477]
 [403 547]
 [328 699]
 [403 547]]
[ 788  950 1027  950]
2
[[126.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [130.   0.   0.   0.   0.]]
arr [126.   0.   0.   0.   0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [130.   0.   0.   0.   0.]
20.0
[[313 478]
 [403 546]
 [328 699]
 [403 546]]
[ 791  949 1027  949]
2
[[392 308]
 [422 474]
 [357 369]
 [424 490]]
[700 896 726 914]
3
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[177  57]
 [174 348]
 [ 49 349]
 [174 348]]
[234 522 398 522]
1
[[359 368]
 [425 489]
 [414 444]
 [414 463]]
[727 914 858 877]
1
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0

[[19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
20.0
[[175  81]
 [170 371]
 [ 45 371]
 [170 372]]
[256 541 416 542]
3
[[ 40   0]
 [179  75]
 [178 145]
 [179  74]]
[ 40 254 323 253]
2
[[19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
20.0
[[ 11   0]
 [177  88]
 [172 388]
 [177  87]]
[ 11 265 560 264]
2
[[1

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[  0 269]
 [179 366]
 [153 564]
 [179 365]]
[269 545 717 544]
2
[[244 392]
 [231 677]
 [241 394]
 [405 491]]
[636 908 635 896]
1
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[  0 269]
 [179 366]
 [153 565]
 [179 365]]
[269 545 718 544]
2
[[244 391]
 [231 677]
 [241 395]
 [405 491]]
[635 908 636 896]
1
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[  0 269]
 [179 366]
 [153 565]
 [179 365]]
[269 545 718 544]
2
[[244 391]
 [231 677]
 [241 395]
 [405 491]]
[635 908 636 896]
1
[[0. 0.

[[244 391]
 [231 676]
 [241 399]
 [405 491]]
[635 907 640 896]
1
[[ 28 599]
 [ 29 658]
 [141 665]
 [ 32 659]]
[627 687 806 691]
2
[[19492. 19460. 19460. 19504. 19586.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]
 [19600. 19600. 19600. 19600. 19600.]]
arr [19492. 19460. 19460. 19504. 19586.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
arr [19600. 19600. 19600. 19600. 19600.]
20.0
[[174 371]
 [153 663]
 [ 48 662]
 [153 664]]
[545 816 710 817]
3
[[  0 269]
 [179 366]
 [154 550]
 [179 365]]
[269 545 704 544]
2
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
arr [0. 0. 0. 0. 0.]
20.0
[[244 391]
 [231 676]
 [242 392]
 [405 491]]
[635 907 634 896]
1
[[ 29 596]
 [ 31 659]
 [141 665]
 [ 32 659]]
[625 690 806 691]
2
[[19469.